# Task 4.9

### Importing libraries and data sets

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
#Creating 'path' shortcut
path = r'C:\Users\Longus\Documents\26-02-2023 Instacart Basket Analysis'

In [3]:
#Importing 'customers.csv'
df_customers = pd.read_csv(os.path.join(path,'02 Data','Original Data','customers.csv'), index_col = False)

In [4]:
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
df_customers.shape

(206209, 10)

In [6]:
df_customers.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [7]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


## Data Wrangling, Data quality and consistency check

In [8]:
#Renaming columns for better readability
df_customers = df_customers.rename(columns = {'First Name' : 'first_name', 'Surnam' : 'surname', 'Gender' : 'gender', 'STATE' : 'state', 'Age' : 'age', 'n_dependants' : 'number_dependants', 'fam_status' : 'family_status'})

In [9]:
df_customers.head()

,user_id,first_name,surname,gender,state,age,date_joined,number_dependants,family_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [10]:
#Checking if there are any missing values (NaN) in the dataframe
df_customers.isnull().sum()

user_id                  0
first_name           11259
surname                  0
gender                   0
state                    0
age                      0
date_joined              0
number_dependants        0
family_status            0
income                   0
dtype: int64

#### There are 11259 missing values in column 'first_name'. First names of our customers are not essential to the analysis, therefore the column will be removed from the dataframe.

In [11]:
#Droping column 'first_name' from the dataframe
df_customers = df_customers.drop(['first_name'], axis = 1)

In [12]:
df_customers.head()

,user_id,surname,gender,state,age,date_joined,number_dependants,family_status,income
0,26711,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [13]:
#Changing data type of 'user_id' from int64 to str/object
df_customers['user_id'] = df_customers['user_id'].astype('str')

In [14]:
df_customers['user_id'].dtype

dtype('O')

In [15]:
#Checking for duplicates
df_customers_no_dups = df_customers[df_customers.duplicated()]

In [16]:
df_customers_no_dups

,user_id,surname,gender,state,age,date_joined,number_dependants,family_status,income


#### No duplicates found

In [17]:
#Checking for mixed data types
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)

#### No mixed data types found

## Combining df_customers with rest of prepared Instacart data

In [18]:
#Importing 'orders_products_merged_aggregated.pkl'
df_ords_prods_merged = pd.read_pickle(os.path.join(path,'02 Data','Prepared Data','orders_products_merged_aggregated.pkl'))

In [19]:
df_ords_prods_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 25 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   order_number            int64   
 3   order_day_of_week       int64   
 4   order_hour_of_day       int64   
 5   days_since_prior_order  float64 
 6   new_customer            bool    
 7   product_id              int64   
 8   add_to_cart_order       int64   
 9   reordered               int64   
 10  _merge                  category
 11  product_name            object  
 12  aisle_id                int64   
 13  department_id           int64   
 14  prices                  float64 
 15  price_range_loc         object  
 16  busiest_day             object  
 17  busiest_days            object  
 18  busiest_period_of_day   object  
 19  max_order               int64   
 20  loyalty_flag            object  
 21  avg_pr

In [20]:
#Changing data type of 'user_id' from int64 to str/object
df_ords_prods_merged['user_id'] = df_ords_prods_merged['user_id'].astype('str')

In [21]:
df_ords_prods_merged['user_id'].dtype

dtype('O')

In [22]:
#Merging 'df_customers' with 'df_ords_prods_merged'
df_ords_prods_custs = df_ords_prods_merged.merge(df_customers, on = 'user_id', how = 'outer')

In [23]:
#Dropping columns 'new_customer' and '_merge' for memory issues
df_ords_prods_custs.drop(['new_customer','_merge'], axis = 1, inplace = True)

In [24]:
df_ords_prods_custs.head(20)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,median_freq,order_freq_flag,surname,gender,state,age,date_joined,number_dependants,family_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
5,3367565,1,6,2,7,19.0,196,1,1,Soda,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
6,550135,1,7,1,9,20.0,196,1,1,Soda,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
7,3108588,1,8,1,14,14.0,196,2,1,Soda,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
8,2295261,1,9,1,16,0.0,196,4,1,Soda,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
9,2550362,1,10,4,8,30.0,196,1,1,Soda,...,20.5,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [25]:
#Exporing the dataframe to .pkl
df_ords_prods_custs.to_pickle(os.path.join(path,'02 Data','Prepared Data','orders_products_customers.pkl'))